In [1]:
from pathlib import Path

import torch as th
import pytorch_lightning as pl

from lightningmodule import MULTModelWarped
from datasets import load_impressionv2_dataset_all

In [2]:
project_name = 'shuffle'
experiement_key = 'f7753cda73dd47399120496c09a5d5d1'

In [3]:
[train_ds, valid_ds, test_ds], target_names = load_impressionv2_dataset_all()
train_dl = th.utils.data.DataLoader(
    train_ds, batch_size=8, pin_memory=True,
)
valid_dl = th.utils.data.DataLoader(
    valid_ds, batch_size=8, pin_memory=True,
)
test_dl = th.utils.data.DataLoader(
    test_ds, batch_size=8, pin_memory=True,
)

In [4]:
checkpoint_dir = Path('logs')/'weights'/f'{experiement_key}_{project_name}'/f'0_{experiement_key}'/'checkpoints'
ckpts = list(checkpoint_dir.glob('*.ckpt'))
assert len(ckpts) == 1
ckpt = ckpts[0]

In [5]:
ck = th.load(ckpt)

In [6]:
from argparse import Namespace
hyp_params = Namespace(**ck['hyper_parameters'])

In [7]:
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [8]:
model = MULTModelWarped(hyp_params, target_names=target_names, early_stopping=None)

In [9]:
model.load_state_dict(ck['state_dict'])

<All keys matched successfully>

In [10]:
trainer.test(model, test_dataloaders=test_dl)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:597: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:605: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor_util.py:106: DeprecationWarning: `np.object` is a

/workspace/transformer/modules/position_embedding.py:23: UserWarning: The number of elements in the out tensor of shape [60] is 60 which does not match the computed number of elements 1526. Note that this may occur as a result of rounding error. The out tensor will be resized to a tensor of shape (1526,). (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/RangeFactories.cu:266.)
  torch.arange(padding_idx + 1, max_pos, out=getattr(make_positions, buf_name))



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1mae': 0.8963922262191772,
 'test_1mae_agreeableness': 0.8885659575462341,
 'test_1mae_conscientiousness': 0.8455855250358582,
 'test_1mae_extraversion': 0.9155563116073608,
 'test_1mae_neuroticism': 0.9099119305610657,
 'test_1mae_openness': 0.9223413467407227,
 'test_loss': 0.017123602330684662}
--------------------------------------------------------------------------------


[{'test_1mae_extraversion': 0.9155563116073608,
  'test_1mae_neuroticism': 0.9099119305610657,
  'test_1mae_agreeableness': 0.8885659575462341,
  'test_1mae_conscientiousness': 0.8455855250358582,
  'test_1mae_openness': 0.9223413467407227,
  'test_loss': 0.017123602330684662,
  'test_1mae': 0.8963922262191772}]

In [11]:
trainer.test(model, test_dataloaders=valid_dl)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1mae': 0.926030695438385,
 'test_1mae_agreeableness': 0.9425502419471741,
 'test_1mae_conscientiousness': 0.9346790313720703,
 'test_1mae_extraversion': 0.9434199929237366,
 'test_1mae_neuroticism': 0.9097769260406494,
 'test_1mae_openness': 0.8997274041175842,
 'test_loss': 0.009370933286845684}
--------------------------------------------------------------------------------


[{'test_1mae_extraversion': 0.9434199929237366,
  'test_1mae_neuroticism': 0.9097769260406494,
  'test_1mae_agreeableness': 0.9425502419471741,
  'test_1mae_conscientiousness': 0.9346790313720703,
  'test_1mae_openness': 0.8997274041175842,
  'test_loss': 0.009370933286845684,
  'test_1mae': 0.926030695438385}]